In [4]:
cd C:/Users/Tanner/Documents/repos/ironmarch-archetypal/

C:\Users\Tanner\Documents\repos\ironmarch-archetypal


In [5]:
import torch
import pandas as pd
import numpy as np
import os
from ast import literal_eval
from scipy import spatial

In [6]:
latent_dim = 16
n_neighbors = 10
embedding_path = 'embeddings\\train_sinkhorn_vae_2021-08-12_19-45-38\\train_sinkhorn_vae_c774e2cc_6_one=0.94555,zero=0.8_2021-08-12_19-45-38\checkpoint_000040\embeddings.csv'

In [7]:
embeddings = pd.read_csv(embedding_path).to_dict(orient = 'records')

def convert_string_to_tensor(string):
    return np.array(string.strip('[').strip(']').replace(',', '').split()).astype('float')

embeddings = [{'id' : item['id'], 'embedding' : convert_string_to_tensor(item['embedding'])} for item in embeddings]

embedding_array = []
id_list = []
for embedding in embeddings:
    embedding_array.append(embedding['embedding'])
    id_list.append(embedding['id'])
embedding_array = np.array(embedding_array)
print(embedding_array.shape)

(217757, 16)


In [8]:
archetypes = np.diag(np.array([1.0 for i in range(latent_dim)]))
distances = spatial.distance_matrix(archetypes, embedding_array)
archetype_closest = np.argsort(distances, axis = 1)
print(archetype_closest)

[[196954 179475 214120 ... 193536 189780 144477]
 [109071 174764  93937 ... 193536 189780 144477]
 [175128 174116 121192 ... 193536 189780 144477]
 ...
 [ 56358   3652 184830 ... 193536 189780 144477]
 [ 71774 193058 124371 ... 193536 189780 144477]
 [105230 207006 118551 ... 193536 189780 144477]]


In [9]:
direct_message_posts = pd.read_csv('iron_march_201911\core_message_posts.csv')
forum_posts = pd.read_csv('iron_march_201911\core_search_index.csv')

In [12]:
from html2text import html2text

def clean_post(post):
    return html2text(post)

def get_post_content(post_id):
    post_location, _, item_id = post_id.split('_')
    item_id = int(item_id)
    if post_location == 'direct':
        pandas_item = direct_message_posts.loc[direct_message_posts['msg_id'] == item_id]
        return clean_post(str(pandas_item['msg_post'].values[0]))
    else:
        pandas_item = forum_posts.loc[forum_posts['index_id'] == item_id]
        return clean_post(str(pandas_item['index_content'].values[0]))

print(get_post_content('direct_messages_1223'))

> I didn't really get that impression from the American nationalists here.
> American_Blackshirt, in _Reaktion_ , explicitly defines pretty much everyone
> currently in the US, Anglo or Continental, white or black, as American,
> except for illegal Hispanic immigrants. Yes, I suppose this is Civic
> Nationalism, but were I an American I'd rather take the option that turned
> the US into an autonomous isolationist state by reviving the things that
> made it great than the one that turned it into a colony again. (However,
> both options are sufficiently far away from the current American mindset
> that I can't see either being realised in the near future.) I'm watching
> your debate with Kacen with great interest.

Why? I think civic nationalism is stupid. It is idealism with no basis in the
material world. Were all just supposed to be united cause were white, black,
brown, cause we're all "Amuurikans!". Where as the ideal of the European is an
ideal rooted in Europeans themselves.

> I'

In [15]:
output_directory = 'post_training_analysis/closest_posts/output'
output_file_name = embedding_path.split('\\')[-1].replace('.csv', '') + '.txt'
output_path = os.path.join(output_directory, output_file_name)

with open(output_path, 'w+', encoding = 'utf-8') as output_file:
    for num, archetype in enumerate(archetype_closest):
        output_file.write(f'---- ARCHETYPE {num} ----\n\n')
        for index in archetype[1: 1 + n_neighbors]:
            item_id = id_list[index]
            try:
                post = get_post_content(item_id)
            except:
                print(item_id)
                raise RuntimeError
            output_file.write(f'MESSAGE {item_id}:\n{post}')